In [166]:
import pandas as pd
import re
import json
import glob
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Define range for file, friends change this help me download when you're free lel
start_int = 0
end_int = 100

In [162]:
## Combine CSV, generate output

# files = glob.glob("Clean_data/*.csv")

# csv = []

# for file in files:
#     df = pd.read_csv(file, index_col=None, header=0)
#     csv.append(df)

# df = pd.concat(csv, axis=0, ignore_index=True)

# df.to_csv("Output/combined_csv.csv")

df = pd.read_csv("Output/combined_csv.csv")

In [163]:
print(df.shape, "\n",df.columns)

(195933, 28) 
 Index(['Unnamed: 0', 'Unnamed: 0.1', 'backers_count', 'blurb', 'category',
       'converted_pledged_amount', 'country', 'created_at', 'current_currency',
       'deadline', 'fx_rate', 'goal', 'id', 'launched_at', 'location', 'name',
       'pledged', 'profile', 'slug', 'source_url', 'spotlight', 'staff_pick',
       'state', 'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type'],
      dtype='object')


In [112]:
urls = df.urls

urls_ = list(map(lambda url:json.loads(url)["web"]["project"],urls))
urls = urls_

In [153]:
path = "Webdriver/chromedriver.exe"
driver = webdriver.Chrome(path)
storys = []

for i in tqdm(range(start_int,end_int)):
    driver.get(urls[i])
    story_raw = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.XPATH,"//div[contains(@class, 'rte__content')]"))
    )
    storys.append(story_raw.text)
driver.quit()

100%|██████████| 100/100 [12:32<00:00,  7.52s/it]


In [184]:
df_export = pd.DataFrame({"index":list(range(0,100)),
                         "story":storys}, index = None)
filename = "Story_" + str(start_int) + "-" + str(end_int)
df_export.to_csv("Output/Story/{}.csv".format(filename))